In [ ]:
# PLEASE: READ ME

#MADCUBA Line Automation
#----------------------
#Version 1
#Copyright (C) 2026 - Jofre Allande González and Víctor Manuel Rivilla. 

#This program is free software: you can redistribute it and/or modify it under
#the terms of the GNU General Public License as published by the Free Software 
#Foundation, either version 3 of the License, or (at your option) any later version.

#This code generates the scripts required to run a macro in MADCUBA. The purpose
#of this macro is to automate the line identification process within the ALMAGAL
#catalog.

#The macro automatically adds all the molecules detected in core 9 of the clump
#AG318.9477-00.1960 (Allande et al., in prep.).

#The code also includes a section that removes all generated files if needed. This
#is useful in case any problem occurs during the macro generation for a given
#folder. If this happens, the files can be safely deleted and generated again.

#Inside the script, there are comments marked as "PLEASE ADD", indicating where the
#user must include custom information such as paths. If these fields are not
#properly filled, the code will not work correctly. Additionally, comments marked
#as "MODIFY IF NEEDED" highlight sections that may require special attention in case
#file names or conventions differ. In such cases, users should modify these parts
#accordingly and adapt them to their own file naming scheme.


In [ ]:
import numpy as np
import os
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.coordinates import SkyCoord
import shutil

In [ ]:
# DEFINE THE DIRECTORY
path = ('/YOURPATH') # PLEASE ADD your path
sources_path = path + '/sources' # PLEASE ADD your folder where the sources are
os.chdir(path) 

In [ ]:
# LOAD TABLE: This table will serve us to know the VLSR of the CLUMPS/CORES
# MODIFY IF NEEDED: This table is from ALMAGAL III (A. Coletta et al., 2025). 
# You can find it here: https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/696/A151#/browse
# "table_ale" is only used to extract the VLSR

table_ale = pd.read_csv( path +'/almagal_prop_sep2023_ALMAGAL_TABLE_COMPLETE.dat',names=['N', 'ALMAGAL_ID', 'ORIG_ID', 'INTERNAL_ID', 'GLON', 'GLAT', 'F70', 'F160', 'F250', 'F350', 'F500', 'VLSR_AG', 'VLSR_ORIG', 'DIST_AG', 'DIST_ORG', 'MASS', 'LUM', 'T_DUST', 'T_BOL', 'SURF_D', 'FLAGS'],skiprows=1,sep=',',on_bad_lines='skip')

In [ ]:
def delete_macros(carpeta_actual):
    directorio_original = os.getcwd()  # Saves the original directory
    try:
        os.chdir(carpeta_actual)  # Changes to the proper directory
        subcarpetas = [subcarpeta for subcarpeta in os.listdir(carpeta_actual) if os.path.isdir(os.path.join(carpeta_actual, subcarpeta))]

        for subcarpeta in subcarpetas:
                print(subcarpeta)
                try:
                    ruta = os.path.join(carpeta_actual, subcarpeta, '/WHERE YOUR FILES WHERE SAVED') # PLEASE ADD: Define a path where the files where saved. If you did not used the macro yet, avoid this message.
                    # Verify if the path exists
                    if os.path.exists(ruta):
                        os.chdir(ruta)  # Change to the specified directory
                        archivos = os.listdir(ruta) # List the files in the folder and delete those that start with "MACRO_SEARCH". You can modify the name with your preferences. 
                        archivos_a_borrar = [archivo for archivo in archivos if archivo.startswith('MACRO_SEARCH')]
                        for archivo in archivos_a_borrar:
                            os.remove(os.path.join(ruta, archivo))  # Delete files
                            print(f"Archivo eliminado: {archivo}")

                        os.chdir(carpeta_actual)  # Return to the main folder
                    else:
                        print(f"The directory does not exist: {ruta}")

                except Exception as e:
                    print(f"Processing error {subcarpeta}: {e}")

    finally:
        os.chdir(directorio_original)  # Return to the original directory when finished

# Example usage:
delete_macros(sources_path) #MODIFY IF NEEDED: You should add the path where your sources are


In [ ]:
def macro_all_molecules_joinedspec(carpeta_actual, table_ale):
    directorio_original = os.getcwd()
    project = 'ALMAGAL'
    config = '7MTM2TM1' 

    molecules = np.array(['CH3CN,V=0','CH3OCHO','CH3CHO','C2H5OH,V=0','CH3OCH3,V=0','CH3OH,VT=0-2','CH3C-13-N,V=0','C2H3CN,V=0','CH3NCO,VB=0','NH2CN',
                        'HC3N*','HOCH2CN','ETHYLENEOXIDE','GGG_GLYCOL','HNCO','CH2(OH)CHO,V=0','CH3O-18-H,V=0-2','HNC-13-O','H2CO','C-13-H3OH,VT=0,1',
                        'CH3COOH,VT=0','CH3COCH3','CH2DOH','CHD2OH,VT=0','C-13-H3OCH3','C-13-H3CHO,VT=0,1','ETHYLFORMATE','CH3OD,VT=0','H2CNH','AGG_GLYCOL',
                        'CH3COOH,VT=1','HDCS','C2H5C-13-N,V=0','CH3CONH2,VT=0,QV'])

    molecules_one_tran = np.array(['T-HCOOH','CO-18','S-33-O','SO,v=0','H2C-13-O','C-13-O','SO2,v=0','H2CO-17','H2CCO,V=0','DCN,v=0','OC-13-S','H2CS','DC(O)NH2','SIO,v=0-10','HC(O)NH2,v12=1',
                                'SiS,v=0-20','t-HC-13-OOH','S-34-O2','HN-15-CO','HC(O)NH2,v=0','HC-13-CCN,v=0','HCC-13-CN,v=0','HCCC-13-N','HCCCN-15,v=0','HCC-13-CN,v7=1','HC-13-(O)NH2',
                                'C2H5CN,v20=1-A',])

    try:
        os.chdir(carpeta_actual)
        subcarpetas = [
            subcarpeta for subcarpeta in os.listdir(carpeta_actual) 
            if os.path.isdir(os.path.join(carpeta_actual, subcarpeta))
        ]

        for subcarpeta in subcarpetas:
            try:
                ruta = os.path.join(carpeta_actual, subcarpeta, '+ WHERE YOU WANT TO SAVE THE FILES') #PLEASE ADD THE PATH WHERE YOU WANT TO SAVE THE FILES
                os.chdir(ruta)
                vlsr = table_ale.loc[table_ale['INTERNAL_ID'] == subcarpeta, 'VLSR_AG'].iloc[0] #MODIFY IF NEEDED
                archivos_a_plotear_spw0 = [
                    archivo for archivo in os.listdir() 
                    if archivo.startswith(subcarpeta + "_spw0_7MTM2TM1_jointdeconv.image.core")  #MODIFY IF NEEDED
                    and archivo.endswith('.spec.fwhm.mean.K.txt') #MODIFY IF NEEDED
                ]
                total_archivos = os.listdir()

                if archivos_a_plotear_spw0:
                    if not any(a.startswith("'MAD_ALMAGAL_") for a in total_archivos): #MODIFY IF NEEDED
                        for archivo in archivos_a_plotear_spw0:
                            core = archivo.replace(subcarpeta + "_spw0_7MTM2TM1_jointdeconv.image.core", "").replace(".spec.fwhm.mean.K.txt", "") #MODIFY IF NEEDED
                            nombre_archivo = "MACRO_SEARCH_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".ijm" #MODIFY IF NEEDED
                            vlsr_str = str(vlsr * 1000)
                            vlsr_orig = str(vlsr)
                            
                            with open(nombre_archivo, "w") as f:
                                f.write(
                                    "// Importing the spectra in ASCII files\n"
                                    "// Include your PATH and change the name of the source\n"
                                    "run(\"Import SPECTRA ASCII FILE\",\" select='" + ruta + "/" + subcarpeta + "_spw0_7MTM2TM1_jointdeconv.image.core" + core + ".spec.fwhm.mean.K.txt'\");\n" #MODIFY IF NEEDED: Name of the spectra spw0
                                    "//\n"
                                    "run(\"Import SPECTRA ASCII FILE\",\" select='" + ruta + "/" + subcarpeta + "_spw1_7MTM2TM1_jointdeconv.image.core" + core + ".spec.fwhm.mean.K.txt'\");\n" #MODIFY IF NEEDED: Name of the spectra spw1
                                    "// Saving both spws in a single MADCUBA spec file\n"
                                    "//\n"
                                    "// In SAVE MASSAJ Change the path and the name of the output file\n"
                                    "//\n"
                                    "run(\"Copy To Other Tab\",\" type=alltab\");\n"
                                    "//\n"
                                    "run(\"Close Tab\",\" nametab=MAD_SPE_" + subcarpeta + "_spw1_7MTM2TM1_jointdeconv.image.core" + core + ".spec.fwhm.mean.K.txt.fits\");\n" #MODIFY IF NEEDED: Name of the spectra spw1
                                    "run(\"Close Tab\",\" nametab=MAD_SPE_" + subcarpeta + "_spw0_7MTM2TM1_jointdeconv.image.core" + core + ".spec.fwhm.mean.K.txt.fits\");\n" #MODIFY IF NEEDED: Name of the spectra spw0
                                    "//\n"
                                    "run(\"Select Rows\",\" rows=ALL alltab=true event=false\");\n"
                                    "run(\"SAVE MASSAJ\",\" type=selected isopen=true select='" + ruta + "/" + "MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config +".spec'\");\n" #MODIFY IF NEEDED: Name to save the .spec file
                                    "run(\"Select Tab\",\" nametab=CopyAllTabs\");\n"
                                    "run(\"Close Tab\",\" nametab=CopyAllTabs\");\n"
                                    "run(\"Select Rows\",\" rows=ALL\");\n"
                                    "//\n"
                                    "// Change the velocity\n"
                                    "run(\"Change Velocity\",\" newvelo=" + vlsr_str + " apply=true\");\n"
                                    "//\n"
                                    "// Search for ethylene glycol (CDMS entry)  using SLIM\n"
                                    "//\n"
                                    "// Select clean transitions to perform the AUTOFIT\n"
                                    "// CHARGE ALL THE MOLECULES OF INTEREST\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HDCS$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HCC-13-CN,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC-13-CCN,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HCCCN-15,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HCC-13-CN,v7=1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$H2CNH$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3OD,vt=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC(O)NH2,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC(O)NH2,v12=1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC-13-(O)NH2$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3COOH,vt=1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C2H5C-13-N,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C2H5CN,v20=1-A$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$ethylformate$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='USER$CH3CONH2,vt=0,Qv$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$aGg_glycol$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3CN,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$CH3OCHO$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$CH3CHO$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C2H5OH,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3OCH3,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3OH,vt=0-2$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C-13-H3OH,v=0,1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3C-13-N,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C2H5CN,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C2H3CN,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3NCO,vb=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$NH2CN$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC3N,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC3N,v7=1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC3N,v7=2$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HC3N,v6=1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"      
									"run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HOCH2CN$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"                                          
									"run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$t-HCOOH$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"                                          
									"run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$Ethyleneoxide$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$CO-18$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$gGg_glycol$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"  
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$S-33-O$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"  
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$HNCO$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$H2C-13-O$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$SO,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C-13-O$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH2(OH)CHO,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3O-18-H,v=0-2$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$HNC-13-O$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$SO2,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$H2CO$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$H2CO-17$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$H2CCO,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$DCN,v=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$HCCC-13-N$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$OC-13-S$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CH3COOH,vt=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$CH3COCH3$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n" 
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$H2CS$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C-13-H3OCH3$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$SiS,v=0-20$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$t-HC-13-OOH$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$S-34-O2$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C-13-H3CHO,vt=0,1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$C-13-H3OH,vt=0,1$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$DC(O)NH2$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$CH2DOH$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$SiO,v=0-10$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='CDMS$CHD2OH,vt=0$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                                    "run(\"SLIM Search\",\" range='selected_data' axislabel='Frequency' axisunit='MHz' molecules='JPL$HN-15-CO$Any$Any$Any$#' searchtype=add datafile='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' datatype=SPECTRA\");\n"
                                    "run(\"SLIM Load Transition\",\" load\");\n"
                            
                                    "//DELETE DIRTY TRANSITIONS - CH3CN \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3CN,v=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220747.2617' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220747.2617' check='false'\");\n"
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220747.2617' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220743.0111' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220743.0111' check='false'\");\n"
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220743.0111' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220730.2611' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220730.2611' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220709.017' check='false'\");\n"        
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220730.2611' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220403.9006' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220403.9006' check='false'\");\n"
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220403.9006' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220403.9006' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220403.9006' check='false'\");\n"
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220403.9006' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220323.6311' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220323.6311' check='false'\");\n"
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220323.6311' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220235.0315' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220235.0315' check='false'\");\n"
                                    "run(\"SLIM Select Rows\", \"rows=ALL\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CN,v=0|1' freq='220235.0315' check='false'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - C-13-H3OCH3 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='C-13-H3OCH3' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='219432.5169' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='219432.5162' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220852.8096' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220852.8373' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='219435.783' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='219434.2391' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='219434.0605' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220852.9322' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220263.6461' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220261.5437' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220261.4568' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='218144.7526' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='218144.6138' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='218143.8147' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220791.9482' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220791.9482' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220791.9479' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220792.664' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220792.3177' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OCH3|1' freq='220792.2944' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH3COOH \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3COOH,vt=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='220432.9309' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='220432.9309' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='217261.8518' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218250.203' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218250.203' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='217265.1446' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='220668.2551' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='220668.2551' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='220668.2551' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='217100.5781' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='219521.5866' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218381.2063' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='220660.927' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='217791.2527' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218044.2146' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218044.2146' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218010.0224' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218010.0224' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218010.0224' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218010.0224' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='217473.8878' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218044.2146' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=0|1' freq='218044.2146' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH3COOH,vt=1 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3COOH,vt=1' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218208.7412' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218326.9942' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='219883.4483' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218208.8768' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217776.9844' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217776.3694' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217875.079' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217875.0739' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218050.1772' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218050.1772' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217607.9632' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217607.9632' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218209.9924' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218209.9924' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217777.153' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217776.2007' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218209.9924' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218209.9924' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217875.0803' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217875.0726' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218050.1772' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='218050.1772' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217607.9632' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COOH,vt=1|1' freq='217607.9632' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - HDCS \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HDCS' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HDCS|1' freq='217263.6893' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HDCS|1' freq='217003.24' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HDCS|1' freq='217005.5' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - C2H5C-13-N,v=0 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='C2H5C-13-N,v=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5C-13-N,v=0|1' freq='220091.1493' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5C-13-N,v=0|1' freq='220195.3113' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5C-13-N,v=0|1' freq='217261.0217' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5C-13-N,v=0|1' freq='218408.6792' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5C-13-N,v=0|1' freq='219592.845' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH3CONH2,vt=0,Qv \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3CONH2,vt=0,Qv' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220273.4299' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220318.5068' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='219957.4648' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218023.0751' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='217438.7031' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='219340.6201' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218774.291' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220228.3304' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220228.3304' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218349.0439' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218349.0439' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220225.4766' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='217582.7467' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='217582.7466' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220228.3304' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220228.3304' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218349.0439' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218349.0439' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='219601.5498' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='219601.5498' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220912.6617' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220232.895' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220056.5533' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220912.6617' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='220912.6617' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='219601.5498' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='219601.5498' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='217582.7467' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='217582.7466' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218454.8312' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CONH2,vt=0,Qv|1' freq='218597.2318' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH3COCH3 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3COCH3' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218760.1534' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218805.9558' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218777.2378' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218747.4423' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219242.1408' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219242.1408' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218091.4115' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219219.931' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220355.3346' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218774.8378' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218091.4483' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218091.4483' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220361.8812' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220361.8812' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219219.9699' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219219.9699' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220367.6372' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220355.3826' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220355.3826' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='217070.5038' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219310.905' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218091.4115' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218162.9289' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219219.931' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220466.3351' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220355.3346' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220368.3229' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218633.8514' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218127.2074' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218127.2074' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218127.2074' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218127.2074' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='217070.5038' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='218162.9289' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220368.3229' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='217022.5087' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='217022.5087' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220361.8812' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220361.8812' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219242.1408' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='219242.1408' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220763.6182' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3COCH3|1' freq='220664.8874' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - DC(O)NH2 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='DC(O)NH2' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='DC(O)NH2|1' freq='218022.1073' check='true'\");\n"
                                    
                                    "//DELETE DIRTY TRANSITIONS - CH3OCHO \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3OCHO' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='218830.592' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='218297.89' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='218280.9' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='218108.438' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='217312.626' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220926.361' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220926.361' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220913.955' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220913.955' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220814.09' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220815.236' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220646.819' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220369.877' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220307.38' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220258.096' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220190.285' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220166.888' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220043.336' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='220030.339' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219827.149' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219822.126' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219763.947' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219705.128' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219695.831' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219642.403' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219622.694' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219584.383' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219568.48' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219479.116' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219483.539' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219411.703' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219417.27' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219194.666' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCHO|1' freq='219154.534' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - HOCH2CN \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HOCH2CN' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='219075.8813' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='219144.2163' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='219451.5218' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='217665.0669' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='218287.8683' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='218287.8683' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HOCH2CN|1' freq='218663.526' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - t-HCOOH \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='t-HCOOH' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='t-HCOOH|1' freq='220037.9671' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - c-C2H4O \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='Ethyleneoxide' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='218335.661' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='218223.5625' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='218335.6536' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='217742.4408' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='217914.7984' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='217615.7' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='217615.7468' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='217333.3949' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='217496.6669' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='Ethyleneoxide|1' freq='219513.2668' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - aGg-glycol \n"
                					"run(\"SLIM Select Molecule\", \"molecule='aGg_glycol' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='220779.684' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='220780.689' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='217848.2997' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218077.1159' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218077.1159' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218126.9284' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218126.9284' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218180.385' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='217844.0092' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218180.385' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='220719.4817' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218238.988' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218304.6705' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218238.988' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218379.9831' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218304.6705' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218238.988' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218379.9831' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218304.6705' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218468.381' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218379.9831' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218574.6802' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218468.381' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218705.81' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218574.6802' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219089.7279' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218705.81' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219385.4256' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219809.4062' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='220496.5917' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219089.72' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219385.1778' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='218371.4955' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219540.4428' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='217450.2702' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='217587.5478' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219764.9258' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='217139.7235' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='219580.6718' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='aGg_glycol|1' freq='217449.9947' check='true'\");\n"
                                                    	
                                    "//DELETE DIRTY TRANSITIONS - C2H5CN,v20=1-A|1 \n"
                					"run(\"SLIM Select Molecule\", \"molecule='C2H5CN,v20=1-A|1' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                					"run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v20=1-A|1' freq='218530.526' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v20=1-A|1' freq='219632.4834' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - C2H3CN \n"
                					"run(\"SLIM Select Molecule\", \"molecule='C2H3CN,v=0' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                					"run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218666.47' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218666.47' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218615.0916' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218585.0717' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218573.6464' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218519.9975' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218463.7385' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218421.8013' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218451.2975' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218406.0146' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218421.8013' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218402.4351' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218402.4513' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218398.5552' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='218398.5554' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='217588.8678' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='220880.958' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='220561.3943' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H3CN,v=0|1' freq='219785.7661' check='true'\");\n"
                				
                                    "//DELETE DIRTY TRANSITIONS - C2H5CN \n"
                					"run(\"SLIM Select Molecule\", \"molecule='C2H5CN,v=0' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                					"run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='218389.97' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='217648.1982' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='217474.2828' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='220667.0949' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='220660.9161' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='219902.4949' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='219505.59' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5CN,v=0|1' freq='219463.6401' check='true'\");\n"
                                    
                                    "//DELETE DIRTY TRANSITIONS - C2H5OH \n"
                					"run(\"SLIM Select Molecule\", \"molecule='C2H5OH,v=0' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='219270.291' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='219392.0557' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='220601.9281' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='217262.3023' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='217496.6653' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='217549.3281' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='217548.1524' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='217803.7431' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='218554.5074' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='218655.9767' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C2H5OH,v=0|1' freq='218654.0892' check='true'\");\n"
                                                    
                                    "//DELETE DIRTY TRANSITIONS - CH3C-13-N \n"
                                   	"run(\"SLIM Select Molecule\", \"molecule='CH3C-13-N' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                					"run(\"SLIM Check Transitions\", \"molecules='CH3C-13-N,v=0|1' freq='220633.8335' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3C-13-N,v=0|1' freq='220599.9797' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3C-13-N,v=0|1' freq='220621.1362' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3C-13-N,v=0|1' freq='220532.3273' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3C-13-N,v=0|1' freq='220570.3728' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3C-13-N,v=0|1' freq='220485.8587' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH3CHO \n"
                					"run(\"SLIM Select Molecule\", \"molecule='CH3CHO' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='217469.2789' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='220446.4939' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='219780.0024' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='219756.5275' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='218218.6856' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='218754.7555' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3CHO|1' freq='217082.9174' check='true'\");\n"
                                                    
                                    "//DELETE DIRTY TRANSITIONS - CH3NCO \n"
                					"run(\"SLIM Select Molecule\", \"molecule='CH3NCO,vb=0' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                					"run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218356.609' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218541.803' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218354.946' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218069.9' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218039.456' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218044.534' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='218002.461' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='217652.088' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='217701.086' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='217595.174' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='217595.174' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='217150.932' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,vb=0|1' freq='217164.881' check='true'\");\n"
                                                 
                                    "//DELETE DIRTY TRANSITIONS - CH3OCH3 \n"
                					"run(\"SLIM Select Molecule\", \"molecule='CH3OCH3,v=0' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                					"run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='218491.914' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='218494.39' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='218489.438' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='218489.438' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='218324.25' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='218324.353' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='217191.4' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='217193.132' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='217189.668' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='217189.669' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='217177.056' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='220893.407' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='220894.974' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='220891.84' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='220848.752' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='220846.542' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='220847.646' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='219465.445' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='219465.61' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OCH3,v=0|1' freq='219301.992' check='true'\");\n"
                                                                                        
                                    "//DELETE DIRTY TRANSITIONS - CH3OH \n"
                					"run(\"SLIM Select Molecule\", \"molecule='CH3OH,vt=0-2' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                					"run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                					"run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='218440.063' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='218273.796' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='218247.359' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='218113.149' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='218113.149' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='217886.504' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='217642.678' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='217642.677' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='217624.026' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='217418.711' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='220892.562' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='220920.857' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='220839.109' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='220886.784' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='220276.728' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='220108.16' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219983.675' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219993.658' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219968.03' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219752.103' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219852.423' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219457.026' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OH,vt=0-2|1' freq='219752.078' check='true'\");\n"
									    
                				    "//DELETE DIRTY TRANSITIONS - NH2CN \n" 
                				    "run(\"SLIM Select Molecule\", \"molecule='NH2CN' component=1 \");\n"
                					"run(\"SLIM Load Transition\", \"load\");\n"
                					"run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='218461.7957' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='218663.6146' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='220489.355' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='220794.9109' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='220126.6435' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='220127.9203' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219914.1758' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='220046.4021' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219875.8275' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219893.7729' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219824.788' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219824.788' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219730.7375' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219730.7375' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219648.6277' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219719.6511' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219648.6277' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219602.1211' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219602.1211' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219473.986' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219445.4938' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219445.4938' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='NH2CN|1' freq='219441.5988' check='true'\");\n"
                                    
                                    "//DELETE DIRTY TRANSITIONS - HNCO\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HNCO' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219798.274' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='220584.751' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219733.85' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219737.193' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219656.7695' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219656.7708' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219547.082' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219547.082' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219392.412' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNCO|1' freq='219392.412' check='true'\");\n"
                                
                                    "//DELETE DIRTY TRANSITIONS - CH2(OH)CHO,v=0\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH2(OH)CHO,v=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='219123.6639' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217611.047' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217608.054' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217603.7471' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='218344.1003' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='219094.9389' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='220055.2262' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='219122.9189' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='219122.9189' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='219303.2158' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217053.7441' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217271.6106' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='220204.0574' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='219230.2337' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217626.1272' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='217830.6967' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='218260.5847' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='220196.8526' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='220196.6583' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2(OH)CHO,v=0|1' freq='220463.8977' check='true'\");\n"
                                    
                                    "//DELETE DIRTY TRANSITIONS - CH3O-18-H\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3O-18-H,v=0-2' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='219429.287' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='219407.809' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220116.744' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220182.163' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220145.91' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220250.951' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220195.139' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220718.048' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='220677.966' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217027.801' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217262.949' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217172.867' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217390.186' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217272.514' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217516.55' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217499.474' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217757.265' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='217969.828' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='218280.872' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='218443.493' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3O-18-H,v=0-2|1' freq='218443.493' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - HNC-13-O\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HNC-13-O' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219804.4386' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219804.4419' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219804.4464' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='220592.6061' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='220592.6127' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='220592.6111' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219740.4505' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219743.7422' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219740.4708' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219743.7623' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219740.4555' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219743.7472' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.6245' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.6255' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.6255' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.6661' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.6671' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.626' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HNC-13-O|1' freq='219663.627' check='true'\");\n"
            
                                    "//DELETE DIRTY TRANSITIONS - SO2,v=0\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='SO2,v=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='SO2,v=0|1' freq='219275.9782' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='SO2,v=0|1' freq='217269.7943' check='true'\");\n"
                                
                                    "//DELETE DIRTY TRANSITIONS - C-13-H3OH,vt=0,1\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='C-13-H3OH,vt=0,1' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OH,vt=0,1|1' freq='217654.81' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OH,vt=0,1|1' freq='217399.55' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OH,vt=0,1|1' freq='217044.616' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OH,vt=0,1|1' freq='220321.807' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3OH,vt=0,1|1' freq='220321.806' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - S-33-O\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='S-33-O' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='S-33-O|1' freq='217832.6422' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='S-33-O|1' freq='217831.7691' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='S-33-O|1' freq='217829.8337' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='S-33-O|1' freq='217827.1782' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - H2CO\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='H2CO' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='H2CO|1' freq='218760.066' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='H2CO|1' freq='218475.632' check='true'\");\n"
                                      
                                    "//DELETE DIRTY TRANSITIONS - GGG-GLYCOL \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='GGG_GLYCOL' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='220249.7871' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='220448.4274' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='220684.6745' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='220766.6623' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='220886.6039' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='217275.9157' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='217539.7182' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='218143.8392' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='gGg_glycol|1' freq='218712.3362' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - t-HC-13-OOH \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='t-HC-13-OOH' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='t-HC-13-OOH|1' freq='219341.8723' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - C-13-H3CHO,vt=0,1 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='C-13-H3CHO,vt=0,1' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='217325.8698' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='218518.8022' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='219979.312' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='217390.703' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='217400.1624' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='220317.3138' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='C-13-H3CHO,vt=0,1|1' freq='220278.6361' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - S-34-O2 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='S-34-O2' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='S-34-O2|1' freq='219496.0008' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='S-34-O2|1' freq='219355.0091' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - DC(O)NH2 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='DC(O)NH2' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='DC(O)NH2|1' freq='218022.1073' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH2DOH \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH2DOH' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='218073.3846' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='218138.971' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='219552.8195' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='218109.5436' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='217447.9073' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='219551.4853' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='218316.3901' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='220552.5864' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH2DOH|1' freq='220071.8051' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - ethylformate \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='ethylformate' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218039.4279' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219569.0035' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219569.0035' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='217888.7785' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='217888.4144' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220205.4025' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220205.3997' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219609.2178' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219609.2178' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219787.93' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219787.9294' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219971.8374' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219971.8213' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219661.3806' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219661.3806' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219826.0913' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219826.0913' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219781.3485' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219859.9843' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219958.265' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219958.2355' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='217888.6434' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='217888.5495' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220205.3856' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220205.3856' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220149.3498' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220148.5623' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220446.9636' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='220431.5374' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219993.4708' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='217052.8222' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218036.5547' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218125.5741' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='219402.4284' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218239.4347' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218744.7183' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218742.6538' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='ethylformate|1' freq='218536.7926' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - HCC-13-CN,v7=1 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HCC-13-CN,v7=1' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HCC-13-CN,v7=1|1' freq='217912.9992' check='true'\");\n"
                                    
                                    "//DELETE DIRTY TRANSITIONS - HC-13-(O)NH2 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HC-13-(O)NH2' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"    
                                    "run(\"SLIM Check Transitions\", \"molecules='HC-13-(O)NH2|1' freq='218189.2' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC-13-(O)NH2|1' freq='220300.8153' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CH3OD,vt=0 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3OD,vt=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='218704.5029' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='220316.0876' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='218760.0837' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='219844.3053' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='218576.9089' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='220228.5633' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='219186.5353' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='218576.8772' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='217132.877' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3OD,vt=0|1' freq='218155.6889' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - CHD2OH,vt=0 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CHD2OH,vt=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CHD2OH,vt=0|1' freq='217491.175' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CHD2OH,vt=0|1' freq='218482.3849' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CHD2OH,vt=0|1' freq='218127.863' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CHD2OH,vt=0|1' freq='218709.0713' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - HC(O)NH2,v12=1 \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HC(O)NH2,v12=1' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC(O)NH2,v12=1|1' freq='218181.68' check='true'\");\n"

                                    "//DELETE DIRTY TRANSITIONS - HC(O)NH2,v=0, \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HC(O)NH2,v=0' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC(O)NH2,v=0|1' freq='219723.2263' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC(O)NH2,v=0|1' freq='219630.5068' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC(O)NH2,v=0|1' freq='219629.3961' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC(O)NH2,v=0|1' freq='218073.9985' check='true'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='HC(O)NH2,v=0|1' freq='218459.213' check='true'\");\n"
                                    
                                    "//DELETE DIRTY TRANSITIONS - CH3NCO,v=0,add \n"
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3NCO,v=0,add' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Check Transitions\", \"type='ALL' check='false'\");\n"
                                    "run(\"SLIM Check Transitions\", \"molecules='CH3NCO,v=0,add|1' freq='218673.183#218682.82' check='true'\");\n"
                                    
                                    "run(\"SLIM Select Molecule\", \"molecule='CH3OCHO' component=1 \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM SIMULATE\", \"molecules='CH3OCHO|1#' logn=15.0 flogn=false tex=60.0 velo=-33.8 fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                    "run(\"SLIM Rotational\", \"action='new' molecules='CH3OCHO' type=integrated option=opt_thinRJ apply=true\");\n"
                                    "logN_rot_CH3OCHO = call('SLIM_Rotational.getValueLogN');\n"
                                    "Texc_rot_CH3OCHO = call('SLIM_Rotational.getValueTex');\n"
                                    "run(\"SLIM SIMULATE\", \"molecules='CH3OCHO|1#' logn=\"+logN_rot_CH3OCHO+\" flogn=false tex=\"+Texc_rot_CH3OCHO+\" velo="+vlsr_orig+" fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                    "//run(\"SLIM Select Molecule\", \"molecule='CH3OCHO' component=1 \");\n"
                                    "//run(\"SLIM Load Transition\", \"load\");\n"
                                    "//run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Estimate LogN\", \"molecule='CH3OCHO|1' setcolumn=true allspecies=false\");\n"
                                    "run(\"SLIM SIMULATE\", \"molecules='CH3OCHO|1#' logn=\"+logN_rot_CH3OCHO+\" flogn=false tex=\"+Texc_rot_CH3OCHO+\" velo="+vlsr_orig+" fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL'\");\n"
                                    "run(\"SLIM AUTOFIT\", \"molecules='CH3OCHO|1#' check=true \");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"

                                    "run(\"SLIM Select Molecule\", \"molecule='HC3N,v=0' component=1 \");\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HC3N,v7=1' component=1 action='add'\");\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HC3N,v7=2' component=1 action='add'\");\n"
                                    "run(\"SLIM Select Molecule\", \"molecule='HC3N,v6=1' component=1 action='add'\");\n"
                                    "run(\"SLIM Load Transition\", \"load\");\n"
                                    "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v6=1' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v7=1' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v7=2' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v=0' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v7=1' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v7=2' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n"
                                    "run(\"SLIM Set New Molecule\", \"molecule='HC3N,v=0' new_name='HC3N*' new_tag='515957' freqs='SELECTED'\");\n")

                                for molecule in molecules:
                                    f.write(
                                        "run(\"SLIM Select Molecule\", \"molecule='"+molecule+"' component=1 \");\n"
                                        "run(\"SLIM Load Transition\", \"load\");\n"
                                        "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                        "run(\"SLIM SIMULATE\", \"molecules='"+molecule+"|1#' logn=\"+logN_rot_CH3OCHO+\" flogn=false tex=\"+Texc_rot_CH3OCHO+\" velo="+vlsr_orig+" fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                        "run(\"SLIM Estimate LogN\", \"molecule='"+molecule+"|1' setcolumn=true allspecies=false\");\n"
                                        "logN_estimated = call('SLIM_Estimate_LogN.getValueLogNLast');\n"
                                        "run(\"SLIM SIMULATE\", \"molecules='"+molecule+"|1#' logn=\"+logN_estimated+\" flogn=false tex=\"+Texc_rot_CH3OCHO+\" velo="+vlsr_orig+" fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                        "run(\"SLIM Load Transition\", \"load\");\n"
                                        "run(\"SLIM Select Rows\", \"rows='ALL'\");\n"
                                        "run(\"SLIM AUTOFIT\", \"molecules='"+molecule+"|1#' check=true \");\n"
                                        "run(\"SLIM Load Transition\", \"load\");\n")
                            
                                for molecule in molecules_one_tran:                               
                                    f.write(
                                        "run(\"SLIM Select Molecule\", \"molecule='"+molecule+"' component=1 \");\n"
                                        "run(\"SLIM Load Transition\", \"load\");\n"
                                        "run(\"SLIM Select Rows\", \"rows='ALL' component='1#'\");\n"
                                        "run(\"SLIM SIMULATE\", \"molecules='"+molecule+"|1#' logn=\"+logN_rot_CH3OCHO+\" flogn=false tex=\"+Texc_rot_CH3OCHO+\" ftex=true velo="+vlsr_orig+" fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                        "run(\"SLIM Estimate LogN\", \"molecule='"+molecule+"|1' setcolumn=true allspecies=false\");\n"
                                        "logN_estimated = call('SLIM_Estimate_LogN.getValueLogNLast');\n"
                                        "run(\"SLIM SIMULATE\", \"molecules='"+molecule+"|1#' logn=\"+logN_estimated+\" flogn=false tex=\"+Texc_rot_CH3OCHO+\" ftex=true velo="+vlsr_orig+" fvelo=false fwhm=5.0 ffwhm=true sourcesize=0.0 fsourcesize=true continuum=false threshold=1.0E-4 typethreshold=intensity \");\n"
                                        "run(\"SLIM Load Transition\", \"load\");\n"
                                        "run(\"SLIM Select Rows\", \"rows='ALL'\");\n"
                                        "run(\"SLIM AUTOFIT\", \"molecules='"+molecule+"|1#' check=true \");\n"
                                        "run(\"SLIM Load Transition\", \"load\");\n")

                                f.write(
                                    "//run(\"SLIM AUTOFIT\", \"molecules='CH3CN,V=0|1#CH3OCHO|1#CH3CHO|1#C2H5OH,V=0|1#CH3OCH3,V=0|1#CH3OH,VT=0-2|1#CH3C-13-N,V=0|1#C2H5CN,V=0|1#C2H3CN,V=0|1#CH3NCO,VB=0|1#NH2CN|1#HC3N*|1#HOCH2CN|1#T-HCOOH|1#ETHYLENEOXIDE|1#CO-18|1#GGG_GLYCOL|1#S-33-O|1#HNCO|1#H2C-13-O|1#SO,V=0|1#C-13-O|1#CH2(OH)CHO,V=0|1#CH3O-18-H,V=0-2|1#HNC-13-O|1#SO2,V=0|1#H2CO|1#C-13-H3OH,VT=0,1|1#H2CO-17|1#H2CCO,V=0|1#DCN,V=0|1#OC-13-S|1#CH3COOH,VT=0|1#CH3COCH3|1#H2CS|1#DC(O)NH2|1#CH2DOH|1#SIO,V=0-10|1#CHD2OH,VT=0|1#HC(O)NH2,V12=1|1#C-13-H3OCH3|1#SIS,V=0-20|1#C-13-H3CHO,VT=0,1|1#T-HC-13-OOH|1#S-34-O2|1#HN-15-CO|1#HC(O)NH2,V=0|1#ETHYLFORMATE|1#CH3OD,VT=0|1#HC-13-CCN,V=0|1#HCC-13-CN,V=0|1#HCCC-13-N,V=0|1#HCCCN-15,V=0|1#HCC-13-CN,V7=1|1#H2CNH|1#HC-13-(O)NH2|1#AGG_GLYCOL|1#C2H5CN,V20=1-A|1#CH3COOH,VT=1|1#HDCS|1#C2H5C-13-N,V=0|1#CH3CONH2,VT=0,QV|1#' \");\n"
                                    "//run(\"SLIM Load Transition\", \"load\");\n"
                                    
                                    "run(\"SLIM Save\", \"namefile='/export/home/magma3/alma_guest/jofre.allande/data/2019.1.00195.L/sources/"+subcarpeta+"/analysis/spectra/7MTM2TM1_newvel/ALMAGAL_"+subcarpeta+"_"+core+"_ETHYLENE.slim'\");\n"
                                    "//run(\"SLIM Save\", \"namefile='/mnt/md0/ialma/mbeltran/alma/ALMAGAL/NEW/SPECTRA/data/2019.1.00195.L/sources/"+subcarpeta+"/analysis/spectra/7MTM2TM1_header/ALMAGAL_"+subcarpeta+"_"+core+"_ETHYLENE.slim'\");\n"
                                    "//run(\"Close Tab\", \"nametab='""MAD_" + project + "_" + subcarpeta + "_" + core + "_" + config + ".fits|Orig.' closeslim=false\");\n"
                                    "//run(\"SLIM Select Tab\", \"tab='SET'\");\n"
                                    "//run(\"SLIM Close\", \"\");\n"
                                    "run(\"Spectra Container\", \"close asksave=false\");\n"
                                )
                                            
                    else:
                        print(f"The file already exists.. CLUMP: {subcarpeta}")

                else:
                    print(f"There is no content in the {subcarpeta} folder.")

            except Exception as e:
                print(f"Does not exist {os.path.join(carpeta_actual, subcarpeta, 'analysis/spectra/7MTM2TM1_newvel')}: {e}")
    finally:
        os.chdir(directorio_original)

macro_all_molecules_joinedspec(sources_path, table_ale) #IN ORDER TO GENERATE THE SCRPITS: 1) ADD THE PATH WHERE THE SOURCES ARE 2)ADD A TABLE WITH THE VLSR OF THE CLUMPS (TABLE ALE in this case)